In [1]:
import pandas as pd
import numpy as np

# other
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
from joblib import dump, load

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use(['seaborn-muted', 'dark_background'])
%matplotlib inline


# processing & model selection
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler

# models
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model

tf.random.set_seed(42)

np.random.seed(410)

2.0.0-rc0


In [3]:
df = pd.read_pickle('../data/ts_pretransform.pkl')

In [4]:
df.head()

,player_name,overall,club,league,nationality,height,weight,intl_rep,dribbling,drib_composure,shooting,passing,defending,physicality,phys_stamina,att_workrate,def_workrate,weak_foot,skill_moves,resource_id,avg_goals,avg_assists,date,days,weekday,gen_pos,promo,source,available,days_release,price_day0,price_day1,price_day2,price_day3,price_day7,y_price
0,Ronaldo,94,Juventus,Serie A TIM,Portugal,187,83,5,90.0,95,93.0,81.0,35.0,79.0,88,High,Low,4,5,20801,0.92,0.37,2018-09-27,8,3,forward,0,packs,0.0,7,1688245,107.052827,96.562821,92.944685,80.951935,2064015.0
1,Ronaldo,94,Juventus,Serie A TIM,Portugal,187,83,5,90.0,95,93.0,81.0,35.0,79.0,88,High,Low,4,5,20801,0.92,0.37,2018-09-28,9,4,forward,0,packs,0.0,8,1774088,95.161289,101.872850,91.890425,74.552390,2095042.0
2,Ronaldo,94,Juventus,Serie A TIM,Portugal,187,83,5,90.0,95,93.0,81.0,35.0,79.0,88,High,Low,4,5,20801,0.92,0.37,2018-09-29,10,5,forward,0,packs,0.0,9,1838174,96.513605,91.843590,98.321160,93.621442,2115861.0
3,Ronaldo,94,Juventus,Serie A TIM,Portugal,187,83,5,90.0,95,93.0,81.0,35.0,79.0,88,High,Low,4,5,20801,0.92,0.37,2018-09-30,11,6,forward,0,packs,0.0,10,1944775,94.518595,91.223303,86.809271,88.388065,2143413.0
4,Ronaldo,94,Juventus,Serie A TIM,Portugal,187,83,5,90.0,95,93.0,81.0,35.0,79.0,88,High,Low,4,5,20801,0.92,0.37,2018-10-01,12,0,forward,0,packs,0.0,11,1999483,97.263893,91.932465,88.727336,78.476986,2119556.0


# Transformation

In [4]:
date_split = datetime(2019, 2, 1)

In [6]:
train_data = df[df.date<date_split]
test_data = df[df.date>=date_split]

In [ ]:
cat_vars = np.array(['club', 'league', 'nationality', 'att_workrate', 'def_workrate', 'gen_pos', 'promo', 'source', 'available'])
num_vars = [x for x in .columns if x not in cat_vars]

id_vars = ['']

In [7]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data), np.array(labels)

,y_price
date,
2018-09-27,2064015.0
2018-09-28,2095042.0
2018-09-29,2115861.0
2018-09-30,2143413.0
2018-10-01,2119556.0
2018-10-02,2154392.0
2018-10-03,2142763.0
2018-10-04,2190325.0
2018-10-05,2094216.0
